In [0]:
# Importação de bibliotecas:

from pyspark.sql.functions import current_timestamp, expr

In [0]:
# Importação dos dados de um banco S3 da Amazon:

df_raw = spark.read.csv(
    "s3a://grao-direto-mmk/raw/grain_logistic_shipping.csv",
    header=True,
    sep=";"
)

# Adicionando coluna de Data e Hora da Carga:

df_raw = (df_raw
      .withColumn('data_hora_carga', 
                  expr('current_timestamp() - INTERVAL 3 HOURS'))
      )

# OBS: Todos os tipos de dados dos arquivos em STR:

display(df_raw)

In [0]:
# Verificação do Schema da importação dos dados:
# A princípio irei estruturar os tipos de dados como STR.

df_raw.printSchema()


In [0]:
# Salvando os dados na camada Bronze do Delta Lake no S3 da Amazon:

df_raw.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("s3a://grao-direto-mmk/bronze/grain_logistic_shipping")

In [0]:
# Criação de um Database Bronze no Delta Lake 

spark.sql("CREATE DATABASE IF NOT EXISTS bronze")

df_raw.write.format("delta")\
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .saveAsTable("bronze.grain_logistic_shipping")


In [0]:
%sql
SELECT * FROM bronze.grain_logistic_shipping
